In [1]:
import numpy as np
import bcolz
import h5py
import os
os.chdir('../..')
import allel
allel.__version__

'1.0.0.dev0'

In [2]:
a = np.arange(20000000)
a

array([       0,        1,        2, ..., 19999997, 19999998, 19999999])

In [3]:
c = allel.chunked.bcolztmp_storage.array(a, cparams=bcolz.cparams(cname='zlib', clevel=1, shuffle=True))
c.flush()
c

carray((20000000,), int64)
  nbytes := 152.59 MB; cbytes := 2.41 MB; ratio: 63.28
  cparams := cparams(clevel=1, shuffle=True, cname='zlib', quantize=0)
  chunklen := 131072; chunksize: 1048576; blocksize: 131072
  rootdir := '/tmp/scikit_allel_5y9lqsv4.bcolz'
  mode    := 'w'
[       0        1        2 ..., 19999997 19999998 19999999]

In [4]:
!du -hs {c.rootdir}

1.9M	/tmp/scikit_allel_5y9lqsv4.bcolz


In [5]:
allel.chunked.ChunkedArrayWrapper(c)

<ChunkedArrayWrapper shape=(20000000,) dtype=int64 chunks=(131072,)
   nbytes=152.6M cbytes=2.4M cratio=63.3
   compression=blosc compression_opts=cparams(clevel=1, shuffle=True, cname='zlib', quantize=0)
   values=bcolz.carray_ext.carray>

In [6]:
h = allel.chunked.hdf5tmp_storage.array(a, compression='gzip', compression_opts=1, shuffle=True, chunks=(c.chunklen,))
h.file.flush()
h

<HDF5 dataset "data": shape (20000000,), type "<i8">

In [7]:
!du -h {h.file.filename}

932K	/tmp/scikit_allel_mo8d55nc.h5


In [8]:
allel.chunked.ChunkedArrayWrapper(h)

<ChunkedArrayWrapper shape=(20000000,) dtype=int64 chunks=(131072,)
   nbytes=152.6M cbytes=920.4K cratio=169.8
   compression=gzip compression_opts=1
   values=h5py._hl.dataset.Dataset>

In [9]:
z = allel.chunked.copy(a, storage='zarrmem')
z

Array((20000000,), int64, chunks=(39063,), order=C)
  nbytes: 152.6M; nbytes_stored: 2.3M; ratio: 66.1; initialized: 512/512
  compressor: Blosc(cname='lz4', clevel=5, shuffle=1)
  store: DictStore

In [10]:
z = allel.chunked.copy(a, storage='zarrtmp')
z

Array((20000000,), int64, chunks=(39063,), order=C)
  nbytes: 152.6M; nbytes_stored: 2.3M; ratio: 66.1; initialized: 512/512
  compressor: Blosc(cname='lz4', clevel=5, shuffle=1)
  store: TempStore

In [11]:
z.store.path

'/tmp/scikit_allel_y0qqf9ak.zarr'

In [12]:
z = allel.chunked.copy(a, storage='zarr', store='array.zarr', overwrite=True)
z

Array((20000000,), int64, chunks=(39063,), order=C)
  nbytes: 152.6M; nbytes_stored: 2.3M; ratio: 66.1; initialized: 512/512
  compressor: Blosc(cname='lz4', clevel=5, shuffle=1)
  store: DirectoryStore

In [13]:
z.store.path

'/home/aliman/src/github/cggh/scikit-allel/array.zarr'

In [14]:
g = allel.chunked.hdf5tmp_storage.table([a, a], compression='gzip', compression_opts=1, shuffle=True)
g

<HDF5 file "scikit_allel_7rxbjikn.h5" (mode r+)>

In [15]:
allel.chunked.ChunkedTableWrapper(g)

<ChunkedTableWrapper shape=(20000000,) dtype=[('f0', '<i8'), ('f1', '<i8')]
   nbytes=305.2M cbytes=1.8M cratio=169.8
   values=h5py._hl.files.File>

In [16]:
allel.chunked.copy_table(g, storage='bcolz')

ctable((20000000,), [('f0', '<i8'), ('f1', '<i8')])
  nbytes: 305.18 MB; cbytes: 7.19 MB; ratio: 42.44
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
[(0, 0) (1, 1) (2, 2) ..., (19999997, 19999997) (19999998, 19999998)
 (19999999, 19999999)]

In [21]:
z = allel.chunked.copy_table(g, storage='zarr')
z

In [22]:
z.grp

Group(/, 2)
  arrays: 2; f0, f1
  store: DictStore

In [23]:
z = allel.chunked.copy_table(g, storage='zarr', store='test.zarr', overwrite=True)
z

In [24]:
z.grp

Group(/, 2)
  arrays: 2; f0, f1
  store: DirectoryStore

In [25]:
!ls -lh test.zarr/

total 24K
drwxrwxr-x 2 aliman aliman 12K Oct 13 10:54 f0
drwxrwxr-x 2 aliman aliman 12K Oct 13 10:54 f1


In [26]:
!ls -lh test.zarr/f0 | head -n10

total 4.0M
-rw------- 1 aliman aliman 4.6K Oct 13 10:54 0
-rw------- 1 aliman aliman 4.6K Oct 13 10:54 1
-rw------- 1 aliman aliman 4.7K Oct 13 10:54 10
-rw------- 1 aliman aliman 4.7K Oct 13 10:54 100
-rw------- 1 aliman aliman 4.7K Oct 13 10:54 101
-rw------- 1 aliman aliman 4.7K Oct 13 10:54 102
-rw------- 1 aliman aliman 4.7K Oct 13 10:54 103
-rw------- 1 aliman aliman 4.7K Oct 13 10:54 104
-rw------- 1 aliman aliman 4.7K Oct 13 10:54 105
ls: write error: Broken pipe


In [27]:
z = allel.chunked.copy_table(g, storage='zarrtmp')
z.grp

Group(/, 2)
  arrays: 2; f0, f1
  store: TempStore

In [28]:
z['f0'].store.path

'/tmp/scikit_allel_gi4s2x_e.zarr'

In [30]:
allel.chunked.ChunkedTableWrapper(z)

<ChunkedTableWrapper shape=(20000000,) dtype=[('f0', '<i8'), ('f1', '<i8')]
   nbytes=305.2M cbytes=4.6M cratio=66.1
   values=allel.chunked.storage_zarr.ZarrTable>